In [1]:
%run setup.ipynb

In [2]:
import os
folder_path = f'{DATA_PATH}'
all_files = os.listdir(folder_path)
dataframes = {}
# Iterating through all the files
for file in all_files:
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        # Loading the CSV and saving it to the dictionary
        current_df = pd.read_csv(file_path, encoding = "latin-1", header = [0,1])
        dataframes[file] = current_df

We create the dataset by concatenating all the files.   
Then we exclude the year 2017 beacuse we had just missing values on the pollutants variables.

In [4]:
combined_df = pd.DataFrame()
lista = ['012','013','014','015','016','017','018','019','020','097','098','108']
for i in range(len(lista)):
    df_1 = dataframes[f'03_{lista[i]}_2017-2020.csv']
    df_2 = dataframes[f'03_{lista[i]}_2021.csv']
    concat_df = pd.concat([df_1, df_2], axis=0, ignore_index=True)
    combined_df = pd.concat([combined_df, concat_df], axis = 0, ignore_index=True)
    combined_df = combined_df.loc[combined_df['DATE']['YYYY'] != 2017]  

In [5]:
combined_df = combined_df.reset_index(drop=True)

In [6]:
print(combined_df.shape)
combined_df.head()

(15696, 123)


DATE ---    PROV  ---            MET   ---                ...  \
   YYYY  MM DD  COM   GP   A[km2]    TG    TN     TX     HU  ...   
0  2018   1  1  138  552  1199.24  2.67  1.28   3.94  87.43  ...   
1  2018   1  2  138  552  1199.24  5.29 -0.56  12.02  58.93  ...   
2  2018   1  3  138  552  1199.24  3.87 -0.18   8.68  70.64  ...   
3  2018   1  4  138  552  1199.24  5.80  1.94   9.96  83.85  ...   
4  2018   1  5  138  552  1199.24  3.02 -0.03   5.97  90.65  ...   

                  SO2    125  ug.m-3   24h     ---                 >TS  \
                   Av Median StDev.5 Max.2 COMx.11 Min.11 COMn.11 GP.9   
0                2.77   2.74     0.6   6.4    1474   0.32    1449    0   
1                1.82   1.79    0.38  5.01    1474   0.33    1449    0   
2                1.02     1.    0.21  4.35    1577   0.18    1516    0   
3                0.99   0.97    0.24  4.22    1474   0.18    1516    0   
4  2.5500000000000003   2.52    0.59   6.8    1474   0.34    1449    0   

        ---         
  A[km2].10 A[%].9  
0        0.     0.  
1        0.     0.  
2        0.     0.  
3        0.     0.  
4        0.     0.  

[5 rows x 123 columns]

In [7]:
for colonna in combined_df.columns:
    combined_df[colonna] = pd.to_numeric(combined_df[colonna], errors='coerce')

In [8]:
combined_df['DATA'] = combined_df.iloc[:,0].astype(str) + '-' + combined_df.iloc[:,1].astype(str) + '-' + combined_df.iloc[:,2].astype(str)

C:\Users\pierd\AppData\Local\Temp\ipykernel_21148\1592434521.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df['DATA'] = combined_df.iloc[:,0].astype(str) + '-' + combined_df.iloc[:,1].astype(str) + '-' + combined_df.iloc[:,2].astype(str)


In [9]:
combined_df['DATA']=pd.to_datetime(combined_df['DATA'])

In [10]:
combined_df.head()

DATE ---    PROV  ---            MET   ---                ...    125  \
   YYYY  MM DD  COM   GP   A[km2]    TG    TN     TX     HU  ... Median   
0  2018   1  1  138  552  1199.24  2.67  1.28   3.94  87.43  ...   2.74   
1  2018   1  2  138  552  1199.24  5.29 -0.56  12.02  58.93  ...   1.79   
2  2018   1  3  138  552  1199.24  3.87 -0.18   8.68  70.64  ...   1.00   
3  2018   1  4  138  552  1199.24  5.80  1.94   9.96  83.85  ...   0.97   
4  2018   1  5  138  552  1199.24  3.02 -0.03   5.97  90.65  ...   2.52   

   ug.m-3   24h     ---                 >TS       ---              DATA  
  StDev.5 Max.2 COMx.11 Min.11 COMn.11 GP.9 A[km2].10 A[%].9             
0    0.60  6.40  1474.0   0.32  1449.0  0.0       0.0    0.0 2018-01-01  
1    0.38  5.01  1474.0   0.33  1449.0  0.0       0.0    0.0 2018-01-02  
2    0.21  4.35  1577.0   0.18  1516.0  0.0       0.0    0.0 2018-01-03  
3    0.24  4.22  1474.0   0.18  1516.0  0.0       0.0    0.0 2018-01-04  
4    0.59  6.80  1474.0   0.34  1449.0  0.0       0.0    0.0 2018-01-05  

[5 rows x 124 columns]

In [11]:
print(combined_df.iloc[:,13][combined_df['DATA']=='2020-2-29'])

789     NaN
2097    NaN
3405    NaN
4713    NaN
6021    NaN
7329    NaN
8637    NaN
9945    NaN
11253   NaN
12561   NaN
13869   NaN
15177   NaN
Name: (AQI_1, Av), dtype: float64


In [12]:
combined_df.to_csv(f'{RESULTS_PATH}/combined_df.csv', index=False)

## DATASET DESCRIPTION

We have time series data for the 12 provinces of Lombardia, a region in the north of Italy. The daily surveys goes from 1st January 2018 to 31 July 2021   
(365 + 365 + 366 + 212 = 1308 days). In total there are 1308 rows for each province. On the 29-02-2020 and on the 29/30/31 December 2020 days we have no data for pollutants variables. In the dataset there are variables related to the weather, but we will not use them for our analysis. 
The dataset has two headers: on the 1st we have the general info like DATE, PROVINCE, AQI_1, and all the pollutant variables, and on the 2nd we have the syntex measures like Mean, Median, Max, Min, StDeviation.

### Variable description

AQI_1: Air Quality Index surveys. For each day are showed Average, StDeviation, Max, Min, Median, Comx, Comn.

The unit of measure of every pollutant is micrograms on cubic meter.

CO: Carbon monoxide   
NH3: Ammonia    
NMVOC: Non-methane volatile organic compound   
NO2: Nitrogen dioxide   
NO: Nitrogen monoxide   
O3: Ozone   
PANS: Perchlorate acids   
PM10: Atmosphere particulate Matter with a diameter of 10 micrometer or less   
PM2.5: Atmosphere particulate Matter with a diameter of 2.5 micrometer or less   
SO2: Sulfur dioxide   

AQI_2: an other Air Quality Index survey.